# Libraries

In [ ]:
! pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00


In [ ]:
# IMPORT LIBRARIES HERE
import tensorflow as tf
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
! apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 44 not upgraded.
Need to get 28.3 MB of archives.
After this operation, 117 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.3 [595 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblzo2-2 amd64 2.10-2build3 [53.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 squashfs-tools amd64 1:4.5-3

# Make Web Driver Object

In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    options.page_load_strategy = "none"
    driver = webdriver.Chrome(options=options)
    return driver

# Data Gathering

## a. Disease List

In [ ]:
driver = web_driver()

driver.implicitly_wait(5)

NameError: name 'web_driver' is not defined

In [ ]:
def get_shadow_root(driver_object, element):
  return driver_object.execute_script('return arguments[0].shadowRoot', element)

In [ ]:
# Trying to scrap lists of diseases from alodokter.com
url = 'https://www.alodokter.com/penyakit-a-z'
driver.get(url)

NameError: name 'driver' is not defined

In [ ]:
wait = WebDriverWait(driver, 10)
try:
    shadow_host = wait.until(EC.visibility_of_element_located((By.TAG_NAME, 'search-a-z-mobile')))
    container = get_shadow_root(driver, shadow_host)
    # Now you can interact with the elements inside the shadow root
except Exception as e:
    print(f"Error: {e}")
    # Handle the error as needed (e.g., logging, retrying, etc.)

In [ ]:
container

In [ ]:
disease_list_elements = container.find_elements(By.CSS_SELECTOR, ".search-a-z-container > .search-a-z-inner > .search-a-z-letter-container > .search-a-z-letter-inner > .search-a-z-result > ul > li")

In [ ]:
diseases = [element.text for element in disease_list_elements]

In [ ]:
for disease in diseases:
    print(disease)

In [ ]:
def get_text(webelement):
  return webelement.text

In [ ]:
disease_list = []
for li in disease_list_elements :
  disease = get_text(li)
  disease_list.append(disease)

In [ ]:
disease_list[:8]

In [ ]:
disease_text = "\n".join(disease_list)

In [ ]:
with open('disease.txt', 'w') as file :
  file.write(disease_text)

## b. Medicine List

In [ ]:
med_url = 'https://www.alodokter.com/obat-a-z'

In [ ]:
driver_med = web_driver()

driver_med.implicitly_wait(5)

In [ ]:
driver_med.get(med_url)

In [ ]:
shadow_host_med = driver_med.find_element(By.TAG_NAME, 'search-a-z-mobile')
med_container = get_shadow_root(driver_med, shadow_host_med)

In [ ]:
med_list_elements = med_container.find_elements(By.CSS_SELECTOR, ".search-a-z-container > .search-a-z-inner > .search-a-z-letter-container > .search-a-z-letter-inner > .search-a-z-result > ul > li")

In [ ]:
med_list = []
for li in med_list_elements :
  med = get_text(li)
  med_list.append(med)

In [ ]:
med_list[:8]

['Abacavir',
 'Abemaciclib',
 'Abrocitinib',
 'Acalabrutinib',
 'Acarbose',
 'ACE Inhibitor',
 'Acebutolol',
 'Acetazolamide']

In [ ]:
med_text = "\n".join(med_list)

In [ ]:
with open('medicine.txt', 'w') as med_file :
  med_file.write(med_text)

## c. Disease Symptoms

In [ ]:
# Use beautifulsoup
from bs4 import BeautifulSoup
import requests

In [ ]:
disease_1 = disease_list[0] # Take example and see the result
query = 'https://alodokter.com/'+disease_1.lower()

response = requests.get(query)
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
import re
header = soup.find('h3', string=re.compile("Gejala"))

In [ ]:
header

<h3><strong>Gejala Abetalipoproteinemia</strong></h3>

In [ ]:
ul = header.find_next_siblings('ul')
li_s = ul[0].find_all('li')
for li in li_s:
  print(li)

<li>Diare dengan tinja berlemak, pucat, berbusa, dan berbau sangat busuk</li>
<li>Mual dan <a href="https://www.alodokter.com/muntah" target="_blank">muntah</a>
</li>
<li>Perut buncit</li>
<li>Gangguan keseimbangan dan koordinasi gerak tubuh</li>
<li>Tulang belakang bungkuk</li>
<li>Keterlambatan tumbuh kembang</li>
<li>Gangguan penglihatan yang memburuk dari waktu ke waktu, akibat gangguan perkembangan retina</li>
<li><a href="https://www.alodokter.com/ataksia" target="_blank">Ataksia</a></li>
<li>Sulit berbicara</li>
<li>Tremor dan <a href="https://www.alodokter.com/tic" target="_blank">tic</a>
</li>
<li>Lemah otot</li>
<li>Kesemutan dan mati rasa</li>
<li>Gerakan cepat dan berulang pada mata (<a href="https://www.alodokter.com/nistagmus" target="_blank">nistagmus</a>)</li>
<li>Pucat, sering lemas, pusing, dan sering pingsan</li>


In [ ]:
# Put it all
def get_symptoms(disease):

  disease_clean = disease.replace(" ", "-").replace("(", "").replace(")", "").replace(".", "").replace("'", "")
  query = f'https://alodokter.com/{disease_clean.lower()}'

  # Response
  response = requests.get(query)
  soup = BeautifulSoup(response.content, 'html.parser')

  # Get element that contains 'gejala' in its text
  header = soup.find('h3', string=re.compile("Gejala"))

  # If the disease name is valid
  try:
        response = requests.get(query)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        header = soup.find('h3', string=re.compile("Gejala"))
        if header is not None:
            symptom_list = []
            sibling = header.find_next_sibling()
            while sibling and sibling.name != "h3":
                if sibling.name == "ul":
                    li_s = sibling.find_all('li')
                    for li in li_s:
                        symptom_list.append(li.get_text().replace("\xa0", ""))
                sibling = sibling.find_next_sibling()
            return symptom_list if symptom_list else None
  except requests.RequestException as e:
      print(f"Request failed for {disease}: {e}")
  return None

In [ ]:
with open("disease.txt", "r") as file:
    disease_list = [line.strip() for line in file if line.strip()]

In [ ]:
from concurrent.futures import ThreadPoolExecutor

In [ ]:
symptoms_list = []
invalid_diseases = []
valid_diseases = []

with ThreadPoolExecutor() as executor:
    futures = {executor.submit(get_symptoms, disease): disease for disease in disease_list}

    for future in futures:
        disease = futures[future]
        result = future.result()
        if result is not None:
            symptoms_list.append(result)
            valid_diseases.append(disease)
        else:
            invalid_diseases.append(disease)


In [ ]:
for s in symptoms_list:
  for i in s:
    if "\xa0" in i :
      i = i.replace("\xa0", "")

In [ ]:
import pandas as pd

df = pd.DataFrame({"Gejala":symptoms_list})

In [ ]:
df.head()

,Gejala
0,"[Diare dengan tinja berlemak, pucat, berbusa, ..."
1,[Muncul banyak bercak hitam yang melayang pada...
2,[Perdarahan dari vagina pada 20 minggu pertama...
3,[]
4,"[Sembelit, Tubuh mudah lelah, Sulit buang air ..."


In [ ]:
df.tail()

,Gejala
1039,[]
1040,[Xantoma kulit di kelopak mata (xanthelasma pa...
1041,"[Mata terasa gatal, Seperti ada sesuatu yang m..."
1042,"[Kulit terasa kencang, terutama setelah mandi,..."
1043,"[Wajah rata, Jidat lebar, Bagian kornea mata b..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044 entries, 0 to 1043
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Gejala  1044 non-null   object
dtypes: object(1)
memory usage: 8.3+ KB


In [ ]:
valid_diseases = [d for d in disease_list if d not in invalid_diseases]

In [ ]:
df["Penyakit"] = valid_diseases

In [ ]:
df.head()

,Gejala,Penyakit
0,"[Diare dengan tinja berlemak, pucat, berbusa, ...",Abetalipoproteinemia
1,[Muncul banyak bercak hitam yang melayang pada...,Ablasi Retina
2,[Perdarahan dari vagina pada 20 minggu pertama...,Abortus Imminens
3,[],Abses
4,"[Sembelit, Tubuh mudah lelah, Sulit buang air ...",Abses Anus


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044 entries, 0 to 1043
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Gejala    1044 non-null   object
 1   Penyakit  1044 non-null   object
dtypes: object(2)
memory usage: 16.4+ KB


In [ ]:
df.to_csv("Daftar Penyakit Alodokter.csv", index=False)

In [ ]:
with open("invalid_disease.txt", "w") as invalid:
  invalid.write("\n".join(invalid_diseases))

In [ ]:
print(f"Total diseases: {len(disease_list)}")
print(f"Valid diseases: {len(valid_diseases)}")
print(f"Invalid diseases: {len(invalid_diseases)}")
print(f"Diseases with symptoms: {len(symptoms_list)}")

# Ensure lengths match
assert len(valid_diseases) == len(symptoms_list), "Mismatch between valid diseases and symptoms list"


Total diseases: 1285
Valid diseases: 1044
Invalid diseases: 241
Diseases with symptoms: 1044


## d. Medicine Detail

In [ ]:
med_1 = med_list[0]
med_url = 'https://alodokter.com/afatinib'
med_page = requests.get(med_url)
med_soup = BeautifulSoup(med_page.content, 'html.parser')

In [ ]:
print(med_soup.find('td', string=re.compile("Manfaat")).find_next_sibling().text.strip())

Menangani kanker paru jenis non-small cell lung cancer (NSCLC) stadium lanjut atau kanker paru-paru stadium 4 yang sudah menyebar ke organ lain.


In [ ]:
# Put it all together
def get_med_details(med):
  med_clean = med.replace(" ", "-")
  med_clean = med_clean.replace("(", "")
  med_clean = med_clean.replace(")", "")
  med_clean = med_clean.replace(".", "")
  med_clean = med_clean.replace("'", "")

  med_url = 'https://alodokter.com/'+med_clean.lower()
  med_page = requests.get(med_url)
  med_soup = BeautifulSoup(med_page.content, 'html.parser')

  med_detail = med_soup.find('table')

  if med_detail is not None :
    manfaat_el = med_soup.find('td', string=re.compile("Manfaat"))
    kategori_el = med_soup.find('td', string=re.compile("Kategori"))
    golongan_el = med_soup.find('td', string=re.compile("Golongan"))

    if manfaat_el is not None and kategori_el is not None and golongan_el is not None:
      manfaat = manfaat_el.find_next_sibling().text.strip()
      kategori = kategori_el.find_next_sibling().text.strip()
      golongan = golongan_el.find_next_sibling().text.strip()
    else :
      return med

    med_detail_list = [manfaat, kategori, golongan]
    return med_detail_list
  else :
    return med

In [ ]:
meds_detail_list = []
invalid_meds = []
with ThreadPoolExecutor() as executor:
  futures = []
  for med in med_list :
    futures.append(executor.submit(get_med_details, med).result())

  for future in futures :
    if isinstance(future, list) :
      meds_detail_list.append(future)
    else :
      invalid_meds.append(future)

In [ ]:
golongan_list = []
kategori_list = []
manfaat_list = []

In [ ]:
for detail in meds_detail_list:
  golongan_list.append(detail[2])
  kategori_list.append(detail[1])
  manfaat_list.append(detail[0])

In [ ]:
valid_meds = [med for med in med_list if med not in invalid_meds]

In [ ]:
med_df = pd.DataFrame({"Obat":valid_meds,
                       "Golongan":golongan_list,
                       "Kategori":kategori_list,
                       "Manfaat":manfaat_list})

In [ ]:
med_df.head()

,Obat,Golongan,Kategori,Manfaat
0,Abacavir,Obat resep,Antivirus nucleoside reverse transciptase inhi...,Memperlambat perkembangan infeksi HIV
1,Abemaciclib,Obat resep,Penghambat protein kinase,Menangani kanker payudara
2,Abrocitinib,Obat resep,Obat imunosupresan,Mengatasi penyakit dermatitis atopik
3,Acalabrutinib,Obat resep,Penghambat protein kinase,Mengobati leukemia limfostik kronis (LLK) atau...
4,Acarbose,Obat resep,Antidiabetes,Mengontrol kadar gula darah pada penderita dia...


In [ ]:
med_df.to_csv("daftar_obat.csv", index=False)

In [ ]:
with open("invalid_meds.txt", "w") as invalid_med_file:
  invalid_med_file.write("\n".join(invalid_meds))